In [1]:
import cdsapi
import pandas as pd
import xarray as xr
from shapely import geometry
from pathlib import Path
from typing import Union
from datetime import datetime

import sys
sys.path.append('..')
from phase_o_matic.preprocess import get_vapour_partial_pressure, convert_pressure_to_pascals

In [53]:
era = xr.open_dataset('../data/ERA5_2020-01-03T20:00_-116.0_45.0_-115.0_46.0.nc')

In [41]:
import numpy as np
levels = ['1','2','3','5','7','10','20','30','50', '70','100','125',\
    '150','175','200','225', '250','300','350','400','450','500','550','600','650',\
    '700','750','775','800','825', '850','875','900','925','950','975','1000']
levels = np.array(levels).astype('int32')
lats = np.linspace(43, 45, 10).astype('float32')
lons = np.linspace(-116, -115, 10).astype('float32')
ds = xr.Dataset(
    {
        'z': (['longitude', 'latitude','level','time'], np.random.random((10, 10, 37, 1)).astype('float32'), {'units':'m**2 s**-2', 'long_name':'Geopotential', 'standard_name' :'geopotential'}),
        't': (['longitude', 'latitude','level','time'], np.random.random((10, 10, 37, 1)).astype('float32'), {'units' :'K', 'long_name' :'Temperature', 'standard_name' :'air_temperature'}),
        'q': (['longitude', 'latitude','level','time'], np.random.random((10, 10, 37, 1)).astype('float32'), {'units' :'kg kg**-1', 'long_name' :'Specific humidity', 'standard_name' :'specific_humidity'}),
    },  
    coords = {
        "longitude" : (["longitude"], lons, {'units' :'degrees_east', 'long_name' :'longitude'}),
        "latitude" : (["latitude"], lats, {'units' :'degrees_north', 'long_name' :'latitude'}),
        "level" : (["level"], levels, {'units' :'millibars', 'long_name' :'pressure_level'}),
        "time" : (["time"], [pd.to_datetime('2020-01-04T09:00')], {'long_name' :'time'}),
    },
    attrs={'Conventions' :'CF-1.6', 'history' :'2023-05-04 17:03:14 GMT by grib_'}
)

In [44]:
ds = convert_pressure_to_pascals(ds)

Pressure levels already converted to pascals


In [45]:
ds = get_vapour_partial_pressure(ds)

In [46]:
ds

<xarray.Dataset>
Dimensions:    (longitude: 10, latitude: 10, level: 37, time: 1)
Coordinates:
  * longitude  (longitude) float32 -116.0 -115.9 -115.8 ... -115.2 -115.1 -115.0
  * latitude   (latitude) float32 43.0 43.22 43.44 43.67 ... 44.56 44.78 45.0
  * level      (level) int32 100 200 300 500 700 ... 92500 95000 97500 100000
  * time       (time) datetime64[ns] 2020-01-04T09:00:00
Data variables:
    z          (longitude, latitude, level, time) float32 0.9168 ... 0.7662
    t          (longitude, latitude, level, time) float32 0.3923 ... 0.3448
    q          (longitude, latitude, level, time) float32 0.2588 ... 0.3068
    vpr        (longitude, latitude, level, time) float64 35.95 ... 4.158e+04
Attributes:
    Conventions:  CF-1.6
    history:      2023-05-04 17:03:14 GMT by grib_